In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pickle
from pathlib import Path

In [2]:
import torch
from google.protobuf import text_format
from second.utils import simplevis
from second.pytorch.train import build_network
from second.protos import pipeline_pb2
from second.utils import config_tool

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [3]:
config_path = "/home/starlet/GoogleDrive/code/secondpytorch/second/configs/experiment/PP_xyres_16_cycleped_featatt.config"
config = pipeline_pb2.TrainEvalPipelineConfig()
with open(config_path, "r") as f:
    proto_str = f.read()
    text_format.Merge(proto_str, config)
input_cfg = config.eval_input_reader
model_cfg = config.model.second
# config_tool.change_detection_range(model_cfg, [-50, -50, 50, 50])
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## Build Network, Target Assigner and Voxel Generator

In [4]:
ckpt_path = "/home/starlet/GoogleDrive/output/DANet_FULL_r1/voxelnet-92800.tckpt"
net = build_network(model_cfg, measure_time=True).to(device).eval()
net.load_state_dict(torch.load(ckpt_path))
target_assigner = net.target_assigner
voxel_generator = net.voxel_generator

height, width 248 296


## Generate anchors

In [5]:
grid_size = voxel_generator.grid_size
feature_map_size = grid_size[:2] // config_tool.get_downsample_factor(model_cfg)
feature_map_size = [*feature_map_size, 1][::-1]

anchors = target_assigner.generate_anchors(feature_map_size)["anchors"]
anchors = torch.tensor(anchors, dtype=torch.float32, device=device)
anchors = anchors.view(1, -1, 7)

In [6]:
feature_map_size

[1, 248, 296]

## Read KITTI Infos

In [7]:
info_path = input_cfg.dataset.kitti_info_path
root_path = Path(input_cfg.dataset.kitti_root_path)
with open(info_path, 'rb') as f:
    infos = pickle.load(f)

## Load Point Cloud, Generate Voxels

In [8]:
info = infos[564]
v_path = info["point_cloud"]['velodyne_path']
v_path = str(root_path / v_path)
points = np.fromfile(
    v_path, dtype=np.float32, count=-1).reshape([-1, 4])
voxels, coords, num_points,voxel_point_mask, voxel_num = voxel_generator.generate(points, max_voxels=90000).values()


In [9]:
# add batch idx to coords
coords = np.pad(coords, ((0, 0), (1, 0)), mode='constant', constant_values=0)
voxels = torch.tensor(voxels, dtype=torch.float32, device=device)
coords = torch.tensor(coords, dtype=torch.int32, device=device)
num_points = torch.tensor(num_points, dtype=torch.int32, device=device)

In [10]:
anchors.shape, voxels.shape, num_points.shape, coords.shape,

(torch.Size([1, 293632, 7]),
 torch.Size([9942, 100, 4]),
 torch.Size([9942]),
 torch.Size([9942, 4]))

## Detection

In [11]:
net.__dict__.keys()

dict_keys(['training', '_parameters', '_buffers', '_non_persistent_buffers_set', '_backward_hooks', '_forward_hooks', '_forward_pre_hooks', '_state_dict_hooks', '_load_state_dict_pre_hooks', '_modules', 'name', '_sin_error_factor', '_num_class', '_use_rotate_nms', '_multiclass_nms', '_nms_score_thresholds', '_nms_pre_max_sizes', '_nms_post_max_sizes', '_nms_iou_thresholds', '_use_sigmoid_score', '_encode_background_as_zeros', '_use_direction_classifier', '_num_input_features', '_box_coder', 'target_assigner', 'voxel_generator', '_pos_cls_weight', '_neg_cls_weight', '_encode_rad_error_by_sin', '_loss_norm_type', '_dir_loss_ftor', '_diff_loc_loss_ftor', '_dir_offset', '_loc_loss_ftor', '_cls_loss_ftor', '_direction_loss_weight', '_cls_loss_weight', '_loc_loss_weight', '_post_center_range', 'measure_time', '_nms_class_agnostic', '_num_direction_bins', '_dir_limit_offset', '_time_dict', '_time_total_dict', '_time_count_dict'])

In [17]:
example = {
    "anchors": anchors,
    "voxels": voxels,
    "num_points": num_points,
    "coordinates": coords,
}
pred = net(example)[0]

In [24]:
net._time_dict

{'voxel_feature_extractor': 0,
 'middle forward': 0,
 'rpn forward': 0,
 'predict': 0,
 'okok': 0}

In [22]:
net.start_timer("okok")

In [23]:
net.end_timer("okok")

In [26]:
net._time_dict['okok']

0

In [28]:
net._time_count_dict

{'voxel_feature_extractor': 2,
 'middle forward': 2,
 'rpn forward': 2,
 'predict': 2,
 'okok': 1}

In [30]:
net._time_total_dict

{'voxel_feature_extractor': 0.44613075256347656,
 'middle forward': 0.0011856555938720703,
 'rpn forward': 0.03686261177062988,
 'predict': 0.0062863826751708984,
 'okok': 4.521486043930054}

In [33]:
net._time_total_dict['rpn forward'] / net._time_count_dict['rpn forward']

0.01843130588531494

In [35]:
import time
start = time.time()
time.sleep(1)
end = time.time()
print(start, end, end-start)

1633256204.0408084 1633256205.0419421 1.001133680343628


In [36]:
net.get_avg_time_dict()

{'voxel_feature_extractor': 0.22306537628173828,
 'middle forward': 0.0005928277969360352,
 'rpn forward': 0.01843130588531494,
 'predict': 0.0031431913375854492,
 'okok': 4.521486043930054}

avg example to torch time: 4.106 ms  
avg prep time: 19.010 ms  
avg voxel_feature_extractor time = 30.596 ms  
avg middle forward time = 0.979 ms  
avg rpn forward time = 29.202 ms  
avg predict time = 5.105 ms  

avg example to torch time: 2.406 ms  
avg prep time: 10.308 ms  
avg voxel_feature_extractor time = 15.743 ms  
avg middle forward time = 0.600 ms  
avg rpn forward time = 15.226 ms  
avg predict time = 2.785 ms  

avg example to torch time: 2.374 ms  
avg prep time: 9.912 ms  
avg voxel_feature_extractor time = 15.462 ms  
avg middle forward time = 0.559 ms  
avg rpn forward time = 14.882 ms  
avg predict time = 2.667 ms  